In [31]:
from abc import ABC, abstractmethod

class Transaction:
  def __init__(self, montant, libelle):
    self.montant = montant
    self.libelle = libelle

    def __str__(self):
      return f"{self.libelle} de {self.montant}€"


class Compte(ABC):
    def __init__(self, titulaire, solde_initial=0):
        self.titulaire = titulaire
        self.__solde = solde_initial

    @abstractmethod
    def retirer(self, montant):
        pass

    def deposer(self, montant, libelle="Dépôt"):
      if montant > 0:
        self.__solde += montant
        print (f"{libelle} de {montant}€ effectué!" )

    def consulter_solde(self):
        return self.__solde

    def _modifier_solde(self, montant):
        self.__solde += montant

class CompteCourant(Compte):  # Heritage
    def __init__(self, titulaire, solde_initial=0, decouvert=0):
      super().__init__(titulaire, solde_initial)
      self.__decouvert = decouvert
      self.historique = [] # Composition

    def retirer(self, montant):
        if self.consulter_solde() >= montant:
            self._modifier_solde(-montant)
            self.historique.append(Transaction (montant, "Retrait"))
        else:
            print("Solde insuffisant")

class CompteEpargne(Compte):
  def __init__(self, titulaire, solde_initial=0, taux=0.02):
      super().__init__(titulaire, solde_initial)
      self.__taux = taux
      self.historique = []

  def calculer_interets(self):
        interets = self.consulter_solde() * self.__taux

        self.deposer(interets, "Intérêts")
        return interets

  def retirer(self, montant):
          if self.consulter_solde() >= montant:
            self._modifier_solde(-montant)
            self.historique.append(Transaction(montant, "Retrait"))
          else:
            print("Solde insuffisant")

  def virement(self, compte_destination, montant):
          if self.consulter_solde() >= montant:
            self.retirer(montant)
            compte_destination.deposer(montant, "virement reçu")
            self.historique.append(Transaction(montant, "Virement vers " + compte_destination.titulaire))
          else:
            print("Solde insuffisant")













In [32]:
class Banque:
  def __init__(self, nom):
    self.nom = nom
    self.comptes = [] # Aggregation, la banque contient des comptes

  def ajouter_compte(self, compte):
    self.comptes.append(compte)

In [33]:
class Client:
  def __init__(self, nom, prenom):
    self.nom = nom
    self.prenom = prenom
# Agrégation : une liste qui contiendra des objets de type Compte
    self.comptes = {}

  def ajouter_compte(self, label, compte):
    self.comptes[label] = compte

  def retirer(self, label, montant):
    self.comptes[label].retirer(montant)

  def consulter_solde(self, label):
    return self.comptes[label].consulter_solde()

  def deposer(self, label, montant):
    self.comptes[label].deposer(montant)

  def afficher_historique(self, label):
    for transaction in self.comptes[label].historique:
      print(transaction)





In [34]:
moi = Client("Dupont", "Jean")

courant_dupont = CompteCourant(moi, 1000)
moi.ajouter_compte("courant", courant_dupont)

moi.deposer("courant", 500)
moi.retirer("courant", 200)

moi.afficher_historique("courant")

epargne_dupont = CompteEpargne(moi, 2000, taux=0.03)
moi.ajouter_compte("epargne", epargne_dupont)

moi.deposer("epargne", 100)
moi.retirer("epargne", 50)

moi.afficher_historique("epargne")





Dépôt de 500€ effectué!
Dépôt de 100€ effectué!
